In [11]:
import pandas as pd
import json
import itertools
import os
import sys

In [15]:
!pip install --user torchtext

  Using cached torchtext-0.6.0-py3-none-any.whl (64 kB)
     |████████████████████████████████| 752.0 MB 8.3 kB/s  eta 0:00:01    |██▌                             | 59.6 MB 12.4 MB/s eta 0:00:57     |███▌                            | 82.0 MB 25.4 MB/s eta 0:00:27     |████▎                           | 101.2 MB 14.0 MB/s eta 0:00:47     |██████▎                         | 148.5 MB 55.6 MB/s eta 0:00:11     |████████                        | 186.2 MB 56.3 MB/s eta 0:00:11     |██████████████                  | 327.4 MB 12.3 MB/s eta 0:00:35     |██████████████                  | 328.6 MB 61.0 MB/s eta 0:00:07     |██████████████                  | 331.1 MB 61.0 MB/s eta 0:00:07     |██████████████▏                 | 333.8 MB 61.0 MB/s eta 0:00:07     |██████████████▉                 | 348.2 MB 7.6 MB/s eta 0:00:53     |███████████████████▎            | 451.7 MB 7.3 MB/s eta 0:00:41     |██████████████████████████████▌ | 716.0 MB 57.2 MB/s eta 0:00:01
     |████████████████████████████████

In [30]:
import torchtext.data


df_path = './output/expansions_artificial_1.json'

with open(df_path,'r') as f:
    df = json.load(f)

df = list(itertools.chain(*df))
df = pd.DataFrame(df)
df.to_csv('tmp.csv', index=False)

src = ('hypothesis',torchtext.data.Field())
trg = ('generations', torchtext.data.Field())

mt_train = torchtext.data.TabularDataset(
path='tmp.csv', format='csv',
fields=(src, trg))


src[1].build_vocab(mt_train, max_size=80000)
trg[1].build_vocab(mt_train, max_size=40000)

train_iter = torchtext.data.BucketIterator(
    dataset=pos, batch_size=32)

for item in mt_train:
    print(item.generations)

['generation']
['creative', 'in', 'the', 'way', 'that', 'their', 'predecessors', 'had."', '"We', 'were', 'very', 'good', 'at', 'it;', 'everybody', 'was', 'a', 'little', 'bit', 'more', 'in', 'their', 'own', 'zone']
['creative', 'to', 'the', 'best', 'of', 'his', 'ability.', 'I', "wouldn't", 'bet', 'against', 'it', 'though.', 'I', 'want', 'that', 'to', 'be', 'a', 'part', 'of', 'my', 'game,', 'just', 'like']
['creative,', 'inclusiveness', 'and', 'self-reflection.', 'In', 'that', 'sense', 'the', 'work', 'of', 'the', 'DREAMER', 'was', 'almost', 'non-existent.', 'The', 'more']
['creative', 'process"', 'by', 'introducing', 'a', 'unique', 'and', 'natural', 'method', 'of', 'extracting', 'and', 'storing', 'chemical', 'components.', '"If', 'you', 'want', 'to', 'get', 'it', 'done,', 'this']
['creative', 'editing', 'work;', 'including', 'video', 'editing;', 'audio', 'editing;', 'and', 'music-based', 'video', 'editing.', 'There', 'are', 'six', 'areas', 'of', 'the', 'project', 'that', 'are']
['creativ

In [51]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis
import itertools 

pairs_df = 'pairs_debug.csv'

pairs_df = pd.read_csv(pairs_df)

pairs_df = pairs_df.dropna()

pairs_df = pairs_df.iloc[:, 1:]

stats_df = pairs_df[['hypothesis', 'cnn_score']].groupby('hypothesis').agg([np.mean, np.std, np.max, np.min, skew, kurtosis]).reset_index(drop=False)
stats_df.columns = stats_df.columns.droplevel(-2)
stats_df = stats_df.rename(columns={stats_df.columns[0]: 'hypothesis'})
pairs_df = pairs_df.drop_duplicates(subset=['hypothesis', 'characters', 'gpt2_score', 'distance_score'])
pairs_df = pairs_df.merge(stats_df, on='hypothesis', how='outer')
pairs_df = pairs_df.drop('cnn_score',axis=1)
pairs_df = pairs_df[[
    'mean',
    'std',
    'amax',
    'amin',
    'skew',
    'kurtosis',
    'gpt2_score',
    'distance_score',
    'hypothesis',
    'characters'
]]
pairs = pairs_df



pairs = pairs.merge(combinations, on='hypothesis', how='left')

pairs_mirror = pairs.copy()
pairs_mirror = pairs_mirror.drop(['hypothesis_2','characters'], axis=1)

pairs = pairs.merge(pairs_mirror, left_on='hypothesis_2', right_on='hypothesis', how='left', suffixes=('_1', '_2'))


pairs = pairs.loc[:,~pairs.columns.duplicated()]
pairs['distance_score'] = pairs['distance_score_1'] - pairs['distance_score_2']
pairs = pairs[[
    'mean_1',
    'std_1',
    'amax_1',
    'amin_1',
    'skew_1',
    'kurtosis_1',
    'gpt2_score_1',
    'mean_2',
    'std_2',
    'amax_2',
    'amin_2',
    'skew_2',
    'kurtosis_2',
    'gpt2_score_2',
    'distance_score'
]]

pairs = np.array(pairs.values.tolist())
pairs

,mean_1,std_1,amax_1,amin_1,skew_1,kurtosis_1,gpt2_score_1,mean_2,std_2,amax_2,amin_2,skew_2,kurtosis_2,gpt2_score_2,distance_score
0,-0.043316,0.432868,0.566264,-0.915906,-0.493109,-0.197022,35.137844,-0.148697,0.324116,0.283625,-0.693651,-0.161989,-1.071977,27.458302,1
1,-0.043316,0.432868,0.566264,-0.915906,-0.493109,-0.197022,35.137844,-0.148697,0.324116,0.283625,-0.693651,-0.161989,-1.071977,27.458302,1
2,-0.043316,0.432868,0.566264,-0.915906,-0.493109,-0.197022,35.137844,-0.148697,0.324116,0.283625,-0.693651,-0.161989,-1.071977,27.458302,1
3,-0.043316,0.432868,0.566264,-0.915906,-0.493109,-0.197022,35.137844,-0.234039,0.492856,0.516928,-1.059575,-0.358327,-0.693156,29.167744,0
4,-0.043316,0.432868,0.566264,-0.915906,-0.493109,-0.197022,35.137844,-0.234039,0.492856,0.516928,-1.059575,-0.358327,-0.693156,29.167744,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,-0.281366,0.296870,0.246054,-0.781133,-0.132898,-0.456056,28.357620,-0.611392,0.501364,-0.065654,-1.398077,-0.621290,-1.094734,41.089573,0
322,-0.281366,0.296870,0.246054,-0.781133,-0.132898,-0.456056,28.357620,-0.611392,0.501364,-0.065654,-1.398077,-0.621290,-1.094734,41.089573,0
323,-0.281366,0.296870,0.246054,-0.781133,-0.132898,-0.456056,28.357620,-0.611392,0.501364,-0.065654,-1.398077,-0.621290,-1.094734,41.089573,0
324,-0.462914,1.204173,0.724777,-3.632203,-1.965971,3.076584,26.435387,-0.188678,0.645691,0.862436,-0.756284,0.901680,-0.524009,32.731735,-1


In [13]:
import pandas as pd
import os

datasets = '/run/media/user/DADOS/NLP/datasets/hashtag_segmentation/BOUN'

dev_boun = os.path.join(datasets, 'Dev-BOUN')
dev_stanford = os.path.join(datasets, 'Dev-Stanford')
test_boun = os.path.join(datasets, 'Test-BOUN') 
snap = os.path.join(datasets, 'SNAP.Hashtags.Segmented.w.Heuristics') 

data_path = {
    'dev_boun': dev_boun,
    'dev_stanford': dev_stanford,
    'test_boun': test_boun,
    'snap': snap
}

data_path_df = []

for item in data_path.values():
    data_path_df.append(pd.read_csv(item, header=None))
    
data_path_df = pd.concat(data_path_df)
assert(data_path_df.shape[0] == data_path_df.drop_duplicates().shape[0])

AssertionError: 

In [3]:
data_path_df = []
for key, value in data_path.items():
    df = pd.read_csv(value, header=None)
    df['source'] = key
    data_path_df.append(df)

data_path_df = pd.concat(data_path_df)
data_path_df.groupby('source').count()

,0
source,
dev_boun,500
dev_stanford,1012
snap,803307
test_boun,499


In [7]:
mask = data_path_df['source'].isin(['dev_boun', 'dev_stanford'])
save_path_df = os.path.join(datasets, 'Dev-BOUN_and_Dev-Stanford')
data_path_df[mask].sort_values(by=0)[0].to_csv(save_path_df, index=False, header=False)

In [126]:
data_path_df.drop_duplicates(subset=0).groupby('source').count()

,0
source,
dev_boun,500
dev_stanford,1012
snap,803031
test_boun,499


In [127]:
data_path_df.drop_duplicates(subset=0, keep=False).groupby('source').count()

,0
source,
dev_boun,500
dev_stanford,737
snap,803031
test_boun,498


In [129]:
mask = ~data_path_df['source'].isin(['snap'])
data_path_df[mask].drop_duplicates(subset=0, keep=False).groupby('source').count()

,0
source,
dev_boun,500
dev_stanford,1012
test_boun,499


In [142]:
clean_df = data_path_df.drop_duplicates(subset=0, keep=False)
mask = clean_df['source'].str.match('snap')
clean_df = clean_df[mask]
clean_df = clean_df.sort_values(0)
clean_df[0].to_csv(os.path.join(datasets, 'clean.SNAP.Hashtags.Segmented.w.Heuristics'), index=False, header=False)

In [153]:
def get_two_samples(series, individual_size, target, prefix='clean.SNAP.Hashtags.Segmented.w.Heuristics'):
    total_size = individual_size * 2
    sample = series.sample(total_size)
    filename_1 = 'sample_{0}_{1}_1'.format(individual_size, prefix)
    filename_2 = 'sample_{0}_{1}_2'.format(individual_size, prefix)
    filename_1 = os.path.join(target, filename_1)
    filename_2 = os.path.join(target, filename_2)
    sample.head(individual_size).to_csv(filename_1, index=False, header=False)
    sample.tail(individual_size).to_csv(filename_2, index=False, header=False)
    

clean_df_series = clean_df[0]
get_two_samples(clean_df_series, 100, datasets)
get_two_samples(clean_df_series, 500, datasets)
get_two_samples(clean_df_series, 1000, datasets)
get_two_samples(clean_df_series, 5000, datasets)
get_two_samples(clean_df_series, 10000, datasets)

In [11]:
import pandas as pd
from collections import defaultdict

path = 'output/evaluation.csv'

df = pd.read_csv(path)
df['characters'] = df['left_hypothesis'].str.replace(" ","")

In [97]:
class WordSegmentationHypothesis(object):
    
    def __init__(self, hypothesis, pair_list):
        self.hypothesis = hypothesis
        self.pair_list = pair_list
    
    def better_than_other(self, other):
        for item in self.pair_list:
            if item['good'] == self.hypothesis and item['bad'] == other.hypothesis:
                return True
        return False
    
    def __lt__(self, other):
        # x < y
        compare = self.better_than_other(other)
        if compare:
            return False
        else:
            return True
        
    def __le__(self, other):
        # x <= y
        compare = self.better_than_other(other)
        if compare:
            return False
        else:
            return True        
        
    def __eq__(self, other):
        # x == y
        return False
        
    def __ne__(self, other):
        # x != y
        return True
        
    def __gt__(self, other):
        # x > y
        compare = self.better_than_other(other)
        if compare:
            return True
        else:
            return False
        
        
    def __ge__(self, other):
        # x >= y
        compare = self.better_than_other(other)
        if compare:
            return True
        else:
            return False      

def record_to_ranking(record):
    if record['left_gpt2_score'] < record['right_gpt2_score']:
        gpt2_ranking = {
            'good': record['left_hypothesis'], 
            'bad' : record['right_hypothesis'] 
        }
    else:
        gpt2_ranking = {
            'bad': record['left_hypothesis'], 
            'good' : record['right_hypothesis']             
        }
        
    if record['predicted_distance_score'] <= 0.0:
        mlp_ranking = {
            'good': record['left_hypothesis'], 
            'bad' : record['right_hypothesis'] 
        }        
    else:
        mlp_ranking = {
            'bad': record['left_hypothesis'], 
            'good' : record['right_hypothesis']             
        }
        
    if record['distance_score'] <= 0.0:
        gold_ranking = {
            'good': record['left_hypothesis'], 
            'bad' : record['right_hypothesis'] 
        }
    else:
        gold_ranking = {
            'bad': record['left_hypothesis'], 
            'good' : record['right_hypothesis']             
        }
        
    ranking = {
        "gpt2_ranking": gpt2_ranking,
        "mlp_ranking": mlp_ranking,
        "gold_ranking": gold_ranking
    }
    return ranking 
        

def sort_pairwise(pairs, model_key='gpt2_ranking'):
    sorted_list = []
    relations_dict = {}
    hypothesis_set = set()
    for pair_item in pairs:
        for status in ['good', 'bad']:
            hypothesis_set.update([pair_item[model_key][status]])
    
    pair_list = []
    for pair_item in pairs:
        pair_list.append(pair_item[model_key])
        
    hypothesis_list = list(hypothesis_set)
    for idx, item in enumerate(hypothesis_list):
        hypothesis_list[idx] = WordSegmentationHypothesis(item, pair_list)
        
    sorted_hypothesis_list = sorted(hypothesis_list, reverse=True)
    
    for idx, item in enumerate(sorted_hypothesis_list):
        sorted_hypothesis_list[idx] = item.hypothesis
        
    return sorted_hypothesis_list


df_items = df.to_dict('records')
characters_dict = defaultdict(list)
                
for item in df_items:
    characters_dict[item['characters']].append(item)

for key, value in characters_dict.items():
    characters_dict[key] = [ record_to_ranking(x) for x in value ]
    
mlp_dict = defaultdict(list)
gpt2_dict = defaultdict(list)
gold_dict = defaultdict(list)

for key, value in characters_dict.items():
    gold_dict[key] = sort_pairwise(value, model_key="gold_ranking")

for key, value in characters_dict.items():
    mlp_dict[key] = sort_pairwise(value, model_key="mlp_ranking")

for key, value in characters_dict.items():
    gpt2_dict[key] = sort_pairwise(value, model_key="gpt2_ranking")

import itertools

unit_size = len(list(gold_dict.values())[0])

hypothesis_to_id = { hypothesis: idx  for idx, hypothesis in enumerate(list(itertools.chain(*gold_dict.values()))) }

gold_dict_ids = [ hypothesis_to_id[hypothesis] for idx, hypothesis in enumerate(list(itertools.chain(*gold_dict.values()))) ]

gpt2_dict_ids = [ hypothesis_to_id[hypothesis] for idx, hypothesis in enumerate(list(itertools.chain(*gpt2_dict.values()))) ]

mlp_dict_ids = [ hypothesis_to_id[hypothesis] for idx, hypothesis in enumerate(list(itertools.chain(*mlp_dict.values()))) ]

gold_dict_ids = [gold_dict_ids[x:x+unit_size] for x in range(0, len(gold_dict_ids), unit_size)]
gpt2_dict_ids = [gpt2_dict_ids[x:x+unit_size] for x in range(0, len(gpt2_dict_ids), unit_size)]
mlp_dict_ids = [mlp_dict_ids[x:x+unit_size] for x in range(0, len(mlp_dict_ids), unit_size)]


In [54]:
gold_dict_ids

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18, 19],
 [20, 21, 22, 23],
 [24, 25, 26, 27],
 [28, 29, 30, 31],
 [32, 33, 34, 35],
 [36, 37, 38, 39],
 [40, 41, 42, 43],
 [44, 45, 46, 47],
 [48, 49, 50, 51],
 [52, 53, 54, 55],
 [56, 57, 58, 59],
 [60, 61, 62, 63]]

In [55]:
gpt2_dict_ids

[[0, 2, 3, 1],
 [4, 7, 5, 6],
 [8, 10, 9, 11],
 [12, 14, 13, 15],
 [17, 16, 18, 19],
 [20, 21, 23, 22],
 [24, 26, 27, 25],
 [28, 29, 30, 31],
 [32, 34, 33, 35],
 [36, 38, 37, 39],
 [40, 41, 42, 43],
 [44, 47, 46, 45],
 [48, 51, 49, 50],
 [52, 55, 53, 54],
 [58, 56, 59, 57],
 [60, 61, 63, 62]]

In [56]:
mlp_dict_ids

[[1, 3, 2, 0],
 [4, 5, 6, 7],
 [9, 10, 11, 8],
 [15, 12, 13, 14],
 [17, 18, 16, 19],
 [21, 20, 22, 23],
 [25, 27, 26, 24],
 [28, 29, 30, 31],
 [32, 33, 34, 35],
 [39, 36, 37, 38],
 [42, 41, 40, 43],
 [45, 44, 47, 46],
 [49, 50, 51, 48],
 [55, 53, 54, 52],
 [57, 56, 58, 59],
 [63, 60, 61, 62]]

In [98]:
import scipy
import numpy as np
compare_gold_gpt2 = list(zip(gold_dict_ids, mlp_dict_ids))

def calculate_kendall(gold_dict_ids, model_dict_ids):
    compare = list(zip(gold_dict_ids, model_dict_ids))
    correlation_array = [ scipy.stats.kendalltau(x,y).correlation for x,y in compare ]
    p_value_array = [ scipy.stats.kendalltau(x,y).pvalue for x,y in compare ]
    average_tau = np.mean(correlation_array)
    std_tau = np.std(correlation_array)
    average_p_value = np.mean(p_value_array)
    std_p_value = np.std(p_value_array)
    result = {
        "average_tau": average_tau,
        "std_tau": std_tau,
        "average_pvalue": average_p_value,
        "std_pvalue": std_p_value
    }
    return result

gold_kendall = calculate_kendall(gold_dict_ids, gold_dict_ids)
gold_kendall['model'] = 'gold'
gpt2_kendall = calculate_kendall(gold_dict_ids, gpt2_dict_ids)
gpt2_kendall['model'] = 'gpt2'
mlp_kendall = calculate_kendall(gold_dict_ids, mlp_dict_ids)
mlp_kendall['model'] = 'mlp'

kendall_df = pd.DataFrame([gpt2_kendall, mlp_kendall, gold_kendall])
kendall_df_columns = (['model'] + kendall_df.columns.values.tolist())[0:-1]
kendall_df = kendall_df[[*kendall_df_columns]]
print(kendall_df.to_string(index=False))

model  average_tau   std_tau  average_pvalue  std_pvalue
 gpt2     0.520833  0.287922        0.546875    0.348569
  mlp     0.229167  0.496078        0.734375    0.457593
 gold     1.000000  0.000000        0.083333    0.000000


In [99]:
better_gpt2_dict = { k: v[0] for k,v in gpt2_dict.items() }
better_mlp_dict = { k: v[0] for k,v in mlp_dict.items() } 
better_gold_dict = { k: v[0] for k,v in gold_dict.items() } 

In [110]:
better_gpt2_dict_df = [ {'characters': k, 'gpt2_hypothesis': v} for k,v in better_gpt2_dict.items() ]
better_mlp_dict_df = [ {'characters': k, 'mlp_hypothesis': v} for k,v in better_mlp_dict.items() ]
better_gold_dict_df = [ {'characters': k, 'gold_hypothesis': v} for k,v in better_gold_dict.items() ]

better_gpt2_dict_df = pd.DataFrame(better_gpt2_dict_df)
better_mlp_dict_df = pd.DataFrame(better_mlp_dict_df)
better_gold_dict_df = pd.DataFrame(better_gold_dict_df)

evaluation_df = better_gpt2_dict_df.merge(better_mlp_dict_df, how='outer', on='characters')
evaluation_df = evaluation_df.merge(better_gold_dict_df, how='outer', on='characters')
assert(evaluation_df.shape[0] == evaluation_df.dropna().shape[0])

from metrics import calculate_recall, calculate_f1, calculate_precision

evaluation_df['gpt2_recall'] = evaluation_df['gpt2_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_recall)
evaluation_df['gpt2_f1'] = evaluation_df['gpt2_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_f1)
evaluation_df['gpt2_precision'] = evaluation_df['gpt2_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_precision)

evaluation_df['mlp_recall'] = evaluation_df['mlp_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_recall)
evaluation_df['mlp_f1'] = evaluation_df['mlp_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_f1)
evaluation_df['mlp_precision'] = evaluation_df['mlp_hypothesis'].combine(evaluation_df['gold_hypothesis'], calculate_precision)

results_df = evaluation_df[['gpt2_recall', 'gpt2_f1', 'gpt2_precision', 'mlp_recall', 'mlp_f1', 'mlp_precision']].agg([np.mean, np.std])
print(results_df.to_string(index=True))

      gpt2_recall   gpt2_f1  gpt2_precision  mlp_recall    mlp_f1  mlp_precision
mean     0.875000  0.875000        0.875000    0.369792  0.350595       0.348958
std      0.341565  0.341565        0.341565    0.397875  0.385453       0.385163


In [17]:
import pandas as pd
from nltk.metrics import edit_distance
df = pd.read_csv('generations.csv')

df['characters'] = df['hypothesis'].str.replace(" ","")
df = df.sort_values(by=['characters', 'gold']).fillna(method='ffill')
df['distance_score'] = df['hypothesis'].combine(df['gold'], edit_distance)
df

,Unnamed: 0,hypothesis,generations,gold,labels,gpt2_score,cnn_score,characters,distance_score
3,0,Batflack,Batflack - 1.0 +0.5 +0.5 +0.5 +0.5 +0.5 +0. Ba...,Batflack,1,36.786156,0.0,Batflack,0
0,2,B at flack,B at flack. He'd probably still get one anyway...,Batflack,0,38.575119,0.0,Batflack,2
1,3,B atflack,B atflack will be used to track the amount of ...,Batflack,0,40.645439,0.0,Batflack,1
2,1,Bat flack,Bat flack for not taking a stand against the U...,Batflack,0,38.336220,0.0,Batflack,1
7,4,Deleanorrie,Deleanorrie The only thing I'm really sure ab...,Deleanorrie,1,42.203606,0.0,Deleanorrie,0
...,...,...,...,...,...,...,...,...,...
119,113,whatadream,whatadreams.com/2016/05/12/further-discover-my...,what a dream,0,29.842560,0.0,whatadream,2
115,116,what else is new,"what else is new here?"" ""We're taking down th...",what else is new,1,26.349754,0.0,whatelseisnew,0
112,119,w hat else is new,"w hat else is new, or a part of the usual band...",what else is new,0,41.919243,0.0,whatelseisnew,1
116,117,what elseis new,"what elseis new?"" ""Yes, I believe that we've ...",what else is new,0,35.583817,0.0,whatelseisnew,1


In [68]:
pairs = df[['hypothesis', 'characters', 'gpt2_score', 'cnn_score', 'distance_score']]

combinations_1 = pairs.groupby('characters')['hypothesis'].apply(lambda x: pd.DataFrame(list(combinations(x,2))))\
        .reset_index(level=1, drop=True)\
        .reset_index()[[0,1]]

combinations_2 = combinations_1[[1,0]]
combinations_2 = combinations_2.rename(columns={
    1: 0,
    0: 1
})
combinations = pd.concat([combinations_1, combinations_2])
combinations['characters'] = combinations[0].str.replace(" ","")
combinations = combinations.sort_values('characters')
combinations = combinations.drop('characters', axis=1)

pairs = pairs.merge(combinations, left_on='hypothesis', right_on=0, how='left')
pairs
pairs = pairs.drop('hypothesis', axis=1)
pairs = pairs.drop('characters', axis=1)
pairs = pairs.rename(columns={
    "gpt2_score" : "left_gpt2_score",
    "cnn_score": "left_cnn_score",
    "distance_score": "left_distance_score",
    0: "left_hypothesis",
    1: "right_hypothesis"
})
pairs = pairs.merge(pairs, left_on='right_hypothesis', right_on='left_hypothesis', how='left')
pairs = pairs.drop("left_hypothesis_y", axis=1)
pairs = pairs.drop("right_hypothesis_y", axis=1)

pairs = pairs.rename(columns={
    "left_gpt2_score_x": "left_gpt2_score",
    "left_cnn_score_x": "left_cnn_score",
    "left_distance_score_x": "left_distance_score",
    "left_hypothesis_x": "left_hypothesis",
    "right_hypothesis_x": "right_hypothesis",
    "left_gpt2_score_y": "right_gpt2_score",
    "left_cnn_score_y": "right_cnn_score",
    "left_distance_score_y": "right_distance_score"
})

pairs["distance_score"] = pairs["left_distance_score"].combine(pairs["right_distance_score"],\
                                                              lambda x, y: x - y)
cols = pairs.columns.values.tolist()
move_first = ['left_gpt2_score', 'left_cnn_score', 'right_gpt2_score', 'right_cnn_score', 'distance_score']
cols = [ x for x in cols if x not in move_first ]
pairs = pairs[[*move_first, *cols]]

,left_gpt2_score,left_cnn_score,right_gpt2_score,right_cnn_score,distance_score,left_distance_score,left_hypothesis,right_hypothesis,right_distance_score
0,36.786156,0.0,38.575119,0.0,-2,0,Batflack,B at flack,2
1,36.786156,0.0,38.575119,0.0,-2,0,Batflack,B at flack,2
2,36.786156,0.0,38.575119,0.0,-2,0,Batflack,B at flack,2
3,36.786156,0.0,38.336220,0.0,-1,0,Batflack,Bat flack,1
4,36.786156,0.0,38.336220,0.0,-1,0,Batflack,Bat flack,1
...,...,...,...,...,...,...,...,...,...
1075,36.924454,0.0,41.919243,0.0,1,2,what elseisnew,w hat else is new,1
1076,36.924454,0.0,41.919243,0.0,1,2,what elseisnew,w hat else is new,1
1077,36.924454,0.0,35.583817,0.0,1,2,what elseisnew,what elseis new,1
1078,36.924454,0.0,35.583817,0.0,1,2,what elseisnew,what elseis new,1


In [10]:
import json

with open('dict.json', 'r') as f:
    dictionary = json.load(f)
    
with open('expansions.json','r') as f:
    expansions = json.load(f)
    
with open('report.json','r') as f:
    report = json.load(f)

In [11]:
import pandas as pd
import itertools
from collections import defaultdict

expansions = itertools.chain(*expansions)
expansions_dict = defaultdict(list)
for item in expansions:
    expansions_dict[item['hypothesis']] += [item['generation']]
expansions = [{'hypothesis': k, 'generations': v} for k,v in expansions_dict.items()]
expansions = pd.DataFrame(expansions)
expansions.head()

,hypothesis,generations
0,Batflack,"[Batflack - 1.0 +0.5 +0.5 +0.5 +0.5 +0.5 +0., ..."
1,Bat flack,[Bat flack for not taking a stand against the ...
2,B at flack,[B at flack. He'd probably still get one anywa...
3,B atflack,[B atflack will be used to track the amount of...
4,Deleanorrie,[Deleanorrie\n\nThe only thing I'm really sure...


In [35]:
report = pd.DataFrame(report)
input_df = expansions.merge(report, on='hypothesis', how='left')
input_df = input_df.drop(['recall', 'precision', 'F1'], axis=1)
input_df['labels'] = input_df['hypothesis'].combine(input_df['gold'], lambda x,y: 1 if x == y else 0)
generations_list = input_df.generations.values.tolist()
labels_list = input_df.labels.values.tolist()
generations_list = [' '.join(x) for x in generations_list]

['Batflack - 1.0 +0.5 +0.5 +0.5 +0.5 +0.5 +0. Batflack and Tuxedo Helm\n\nIt\'s easy to see why this could be a problem, but you can use the tools that can Batflack\n\nThe main event will take place in Toronto at the Queen\'s Park.\n\nThis year, The Art of Canadian Crafts Batflack and the Lighthouse, and then headed for the island of New Mexico.\n\nAfter their expedition had been successful, the astronauts had Batflack\n\nRacine:\n\nTiny T. Stank\n\nR.H.G.\n\nStank\n Batflack #18 (in 3 minutes) Kostya Mokhtarov #9 (in 4 minutes) Kostya Mokhtar Batflack.com Batflack.com\n\nFor further information about the website, please visit: www.pflack.com/pflack. For Batflack and L.A.\'s "Star Wars-Style" T-shirts. They sell out quickly and many of the artists seem to Batflack.\n\nA spokesperson said a spokesperson for the Canadian Association of Medical Examiners (CAIRE) said their team would not',
 'Bat flack for not taking a stand against the US, he\'s seen many of the same things as Mr. Obama: 

In [22]:
dictionary

,hypothesis,score,characters
0,tryingtosleep,37.444962,tryingtosleep
1,t ryingtosleep,55.577179,tryingtosleep
2,tr yingtosleep,56.359470,tryingtosleep
3,try ingtosleep,45.852482,tryingtosleep
4,tryi ngtosleep,53.299782,tryingtosleep
...,...,...,...
60733,some one sn eak me in,59.622208,someonesneakmein
60734,some one sne ak me in,61.105171,someonesneakmein
60735,some one snea k me in,63.600086,someonesneakmein
60736,some one sneak m e in,58.303818,someonesneakmein


In [23]:
expansions

,hypothesis,generation,characters
0,10 RANDOM ACTS,10 RANDOM ACTS: A PROGRAM DUMP IN ORDER TO HAC...,10RANDOMACTS
1,10 RANDOM ACTS,10 RANDOM ACTS AVAILABLE IN PARTICULAR REGISTR...,10RANDOMACTS
2,10 RANDOM ACTS,10 RANDOM ACTS\n\nRANDOM MECHANISMS\n\nRAGE\n\...,10RANDOMACTS
3,10 RANDOM ACTS,10 RANDOM ACTS ON THE MASS PROBLEM OF CHILD CA...,10RANDOMACTS
4,10 RANDOM ACTS,10 RANDOM ACTS\n\n\nF.2. In response to this r...,10RANDOMACTS
...,...,...,...
19995,y ou will pay,y ou will pay on your behalf.\n\n3.) The perso...,youwillpay
19996,y ou will pay,y ou will pay them out of my hands\n\nWELCOME ...,youwillpay
19997,y ou will pay,y ou will pay. I don't care about your money. ...,youwillpay
19998,y ou will pay,"y ou will pay all of the above. However, it's ...",youwillpay
